# Control a Robotic Arm with ROS
 If you wanted to develop a robot, you had to build a complete system: a physical device, of course, but also the control systems, interface, and inspection tools required to get the robot up and running as a test platform.

We have to:
- Create the Robot arm model
- Import the Robot arm model in ROS
- Control the kinematics 

### Install some packages

First of all we need to install:
- urdf tutorial package
- the moveit package.

The simplest way to install MoveIt is from pre-built binaries (Debian):

In [ ]:
rosdep update
sudo apt-get update
sudo apt-get install python3
sudo apt-get install ros-melodic-urdf-tutorial
sudo apt install ros-melodic-moveit

#### Documentation
Interesting links are:
- ROS Tutorial: Create an arm on a mobile robot using Moveit!: https://www.youtube.com/watch?v=l4dtSRvlAjg
- Control any Robotic Arm with ROS: https://blog.usejournal.com/control-any-robotic-arm-with-ros-b10a3115306c
- Udemy course: https://www.udemy.com/course/robotics-with-ros-build-robotic-arm-in-gazebo-and-moveit/
- Udemy course repository: https://github.com/noshluk2/4_Robotic-Arm-Manupilation-using-Gazbo-and-Moveit
- How to control a robot arm with the Arduino and ROS: https://maker.pro/arduino/tutorial/how-to-control-a-robot-arm-with-ros-and-arduino
- Moveit tutorials: http://docs.ros.org/en/melodic/api/moveit_tutorials/html/index.html
- Moveit Setup Assistant Tutorial: http://docs.ros.org/en/melodic/api/moveit_tutorials/html/doc/setup_assistant/setup_assistant_tutorial.html

Public repositories in:
- https://bitbucket.org/theconstructcore/
- https://bitbucket.org/theconstructcore/my-robotic-manipulator/src/master/
- https://bitbucket.org/theconstructcore/two-wheeled-robot-motion-planning/src/master/


## 1. Create the Robot arm model

First of all, we create a "arm_kinematics_ws" where we will install the packages:
- custom_robotic_arm package: where the URDF model of robotic arm is located.
- robot kinematics control packages: 
    - the custom_moveit package: generated authomatically with "moveit setup assistant"
    - the custom_arm_control package: to perform a programmatically kinematics control 

Create a new repository "arm_kinematics_ws" in your github

Syncronise the repository in your local computer

Create the src folder and compile the workspace

### 1.1. Create the URDF model of a robotic arm
We can create an arm model in three different ways:

- From scratch
- From SolidWorks
- From an existing package in ROS

#### 1.1.1. Design a 3D robot arm model from scratch

ROS and MoveIt require our robotic arm model files to be in URDF. The Unified Robotic Description Format (URDF) is an XML file format used in ROS to describe all elements of a robot. It lists all the kinematic properties of the links and joints in a chain from base to tip. ROS also accepts .xacro files which are very similar to urdf and are easier to write.

We can make use of any robot arm design files available on the internet in CAD neutral format (such as STL). Alternatively, one can design it from scratch using any of the popular CAD software packages such as SolidWorks.

First of all you need to create a new package for robot description

In [ ]:
catkin_create_pkg custom_robotic_arm rospy std_msgs

Create "urdf" and "launch" folders inside the new created package.

In the "urdf" folder, we create a file named "custom_arm.urdf", where we are going to describe all the elements of the robot. 

Below, is the code describing one fixed link and its joint, and also another one with a continuous joint.

In [ ]:
<!-- Defining the version and the name of the robot -->
<?xml version="1.0"?>
<robot name="custom_arm">
<!-- First connection to the base link-->
  <link name="world"/>
    <link name="base_link">
        <visual>
        <!-- Geometry and material of the baselink -->
            <geometry>
                <cylinder length="0.05" radius="0.1"/>
            </geometry>
            <material name="silver">
                <color rgba="0.75 0.75 0.75 1"/>
            </material>
            <!-- Location of the base link -->
            <origin rpy="0 0 0" xyz="0 0 0.025"/>
        </visual>
        <!-- Definition of the collision properties -->
        <collision>
            <geometry>
                <cylinder length="0.05" radius="0.1"/>
            </geometry>
                <origin rpy="0 0 0" xyz="0 0 0.025"/>
        </collision>
        <!-- Definition of the intertial properties -->
        <!-- It defines the properties that will act if there is a collision -->
        <inertial>
            <mass value="1.0"/>
            <origin rpy="0 0 0" xyz="0 0 0.025"/>
            <inertia ixx="0.0027" iyy="0.0027" izz="0.005" ixy="0" ixz="0" iyz="0"/>
        </inertial>
        </link>
<!-- Definition of the joint for the link created -->
<!-- This is a fixed link, it is not going to move -->
<!-- Defining the connection between the world and the base link, with a fixed joint -->
  <joint name="fixed" type="fixed">
    <parent link="world"/>
    <child link="base_link"/>
  </joint>

<!-- New link connected to the base one, with a joing that will move -->
<!-- Definition of the geometry and the mechanical properties -->
  <link name="One">
    <visual>
        <geometry>
            <cylinder length="0.5" radius="0.05"/>
        </geometry>
        <material name="silver">
            <color rgba="0.75 0.75 0.75 1"/>
        </material>
        <origin rpy="0 0 0" xyz="0 0 0.25"/>
    </visual>
    <collision>
        <geometry>
            <cylinder length="0.5" radius="0.05"/>
        </geometry>
        <origin rpy="0 0 0" xyz="0 0 0.25"/>
    </collision>
    <inertial>
        <mass value="1.0"/>
        <origin rpy="0 0 0" xyz="0 0 0.25"/>
        <inertia ixx="0.02146" iyy="0.02146" izz="0.00125"
        ixy="0" ixz="0" iyz="0"/>
    </inertial>
  </link>
  <!-- Defining a continuous joint, which can rotate  -->
  <!-- In this case, we have to define the axis and origin -->
  <joint name="One_joint" type="continuous">
    <axis xyz="0 0 1"/>
    <parent link="base_link"/>
    <child link="One"/>
    <origin rpy="0 0 0" xyz="0.0 0.0 0.05"/>
  </joint>

We can repeat this step, as many times as links and joints we want our robot to have. In this case, as it is a simple arm, it is going to have four joints and links. To observe the evolution of the robot, we use RViz with the following command on the terminal.

Then you can see the robot arm using the custom file arm_rviz.launch

In [ ]:
roslaunch custom_robotic_arm arm_rviz.launch

The custom launch file for this purpose (arm_rviz.launch) is:

In [ ]:
<launch>
  <arg name="rvizconfig" default="$(find custom_robotic_arm)/rviz/urdf.rviz" />
  <param name="robot_description" textfile='$(find custom_robotic_arm)/urdf/custom_arm.urdf'/>
  <node name="robot_state_publisher" pkg="robot_state_publisher" type="robot_state_publisher" />
  <node name="jsp_gui" pkg="joint_state_publisher_gui" type="joint_state_publisher_gui" />
  <node name="rviz" pkg="rviz" type="rviz" args="-d $(arg rvizconfig)" required="true" />
</launch>

![Getting Started](./Images/1_arm_rviz2.png)

#### 1.1.2. Generate URDF model file from SolidWorks
Fortunately, due to a few talented ROS developers, there exists a convenient plugin in Solidworks to export our assembly in URDF format.

Assuming you have your model completely set up as a SolidWorks assembly, we now use the SolidWorks to URDF Exporter: http://wiki.ros.org/sw_urdf_exporter

Follow the tutorial to obtain the URDF file: https://blog.usejournal.com/control-any-robotic-arm-with-ros-b10a3115306c

The built package will contain directories for meshes, textures and robots. It will also contain a ROS package.xml (manifest) file so you can use this as a ROS package by just copying it to your ROS system. The path locations in the URDF are relative to the package itself. Use proper ROS package names to avoid errors.

Once created the URDF file, we can generate the corresponding package in src folder

In [ ]:
catkin_create_pkg dexarm rospy std_msgs

#### 1.1.3. Import the Robot arm model in ROS

Install the robot_description package of "panda robot arm" already available in ROS Melodic using this command:

In [ ]:
sudo apt-get install ros-melodic-franka-description

Copy the folder "franka_description" located in "/opt/ros/melodic/share/" to our src folder

Compile again the workspace

Verify the lasts lines in .bashrc file (change to your repository path)

In [ ]:
source /opt/ros/melodic/setup.bash
source /media/sf_github_manelpuig/arm_kinematics_ws/devel/setup.bash

## 1.2. Spawn robot model in Gazebo

A robot in Gazebo needs a proper control transmission. So, we need to create transmissions and actuators in the joints of the robot.

First we have to define the transmissions and controllers to the URDF model. The syntax for 1 transmission is:

In [ ]:
                    <!--TRANSMISSION-->
<!--ONE-->
<transmission name ="Trans_1">
    <type>transmission_interface/SimpleTransmission</type>
    <joint name="One_joint">
        <hardwareInterface>hardware_interface/PositionJointInterface</hardwareInterface>
    </joint>
    <actuator name="Motor1">
        <hardwareInterface>hardware_interface/PositionJointInterface</hardwareInterface>
        <mechanicalReduction>1</mechanicalReduction>
    </actuator>
</transmission>

A robot need also a controller, so we need a Gazebo plugin that controls the transmission and actuations. We come back to "custom_arm.urdf", and we write the following:

In [ ]:
<gazebo>
    <plugin name="control" filename="libgazebo_ros_control.so"/>
</gazebo>

Now you need to define the controller parameters file (controller.yaml) in a config folder as follows:

In [ ]:
arm_controller:
  type: position_controllers/JointTrajectoryController
  joints:
    - One_joint
    - Two_joint
    - Three_joint
    - Four_joint

If you want to specify the PID motor control parameters you need to create a gazebo_ros_control_params.yaml with the PID values. Test proper values for each robot arm and joint. There is a set of proper values for our custom robot arm in the yaml created file.

Now we can create a "arm_gazebo.launch" file to spawn the robot arm and control the joint angles:

In [ ]:
<launch>
    <param name="robot_description" textfile="$(find custom_robotic_arm)/urdf/custom_arm.urdf"/>
    <!-- To specify the PID motor parameters /-->
    <!--rosparam file="$(find custom_robotic_arm)/config/gazebo_ros_control_params.yaml" command="load"/-->
    <include file="$(find gazebo_ros)/launch/empty_world.launch" />
    <node name="spawn_urdf" pkg="gazebo_ros" type="spawn_model" args="-param robot_description -urdf -model custom_arm"    />
    <rosparam file ="$(find custom_robotic_arm)/config/controller.yaml"/>
    <node name= "controller_spawner" pkg= "controller_manager" type="spawner" args="arm_controller"/>
    <node name="robot_state_publisher" pkg="robot_state_publisher" type="robot_state_publisher"/>
    <node name="joint_state_publisher" pkg="joint_state_publisher" type="joint_state_publisher"/>
</launch>

In [ ]:
roslaunch custom_robotic_arm arm_gazebo.launch 

We can specify the joint angles to speciffic values publishing to the topic /arm_controller/command trajectory_msgs/JointTrajectory

In [ ]:
rostopic pub /arm_controller/command trajectory_msgs/JointTrajectory '{joint_names:["One_joint","Two_joint","Three_joint","Four_joint"],points:[{positions:[0.1,-0.5,0.5,0.75],time_from_start:[1,0] }]}'

![Getting Started](./Images/1_arm_gazebo1.png)

Remember the instruction to properly close the gazebo:

killall gzserver && killall gzclient

## 1.3. Create a moveit_config package

The quickest way to get started using MoveIt is through its RViz plugin. Rviz is the primary visualizer in ROS and an incredibly useful tool for debugging robotics.

http://docs.ros.org/en/melodic/api/moveit_tutorials/html/doc/quickstart_in_rviz/quickstart_in_rviz_tutorial.html
http://docs.ros.org/en/melodic/api/moveit_tutorials/html/doc/setup_assistant/setup_assistant_tutorial.html

To start the MoveIt Setup Assistant:

In [ ]:
roslaunch moveit_setup_assistant setup_assistant.launch

Follow the "Moveit Setup Assistant" Tutorial to create the moveit_config package for the panda robot arm

http://docs.ros.org/en/melodic/api/moveit_tutorials/html/doc/setup_assistant/setup_assistant_tutorial.html

You have now the panda_moveit_config package properly installed

Carefully:
- Choose the robot model created on the previous sections, "custom_arm.urdf".
- Generate Collision Matrix
- no virtual joints
- add the joints to "Planning groups": arm (all joints) and hand (last Four-joint)). Select RRT Planner
- add a tool with end effector group "hand" and parent link Four
- no passive joints
- no ROS control
- 3D perception to point cloud
- You can obtain the urdf file generated automatically
- save to custom_moveit package name

In [ ]:
roslaunch moveit_arm demo.launch

You can make the robot do some random movements. On the section 'MotionPlanning', in the tab 'Planning', you can select a random goal state. You 'Plan' the movement, and then 'Execute'. Then, you see the robot start to move.

![Getting Started](./Images/1_arm_moveit1.png)

## 1.4. Building Robot Arm Controller
Now we are going to make a synchronization between Moveit and Gazebo. When we plan a robot movement in rviz, we want to obtain the same movement in the real world (gazebo in our case).

We create a file "controller.yaml" on the 'config' folder of moveit package. With this file we will enable moveIt to use the controller defined previously to work with gazebo.

In [ ]:
controller_manager_ns: /
controller_list:
  - name: arm_controller
    action_ns: follow_joint_trajectory
    type: FollowJointTrajectory
    joints:
      - One_joint
      - Two_joint
      - Three_joint
      - Four_joint

Now, we need to change the file "custom_arm_moveit_controller_manager.launch.xml". It is going to use the controller previously defined for Gazebo and link it with Moveit . When we move the robot in Moveit, it will move in Gazebo also.

Note that we have to add:
- param name="controller_manager_name" value="/"
- param name="use_controller_manager" value="true"

and address to the controller.yaml created before
- rosparam file="$(find custom_moveit)/config/controller.yaml"

In [ ]:
<launch>
  <!-- loads moveit_controller_manager on the parameter server which is taken as argument 
    if no argument is passed, moveit_simple_controller_manager will be set -->
  <arg name="moveit_controller_manager" default="moveit_simple_controller_manager/MoveItSimpleControllerManager" />
  <param name="moveit_controller_manager" value="$(arg moveit_controller_manager)"/>
  <param name="controller_manager_name" value="/" />
  <param name="use_controller_manager" value="true" />
  <!-- loads ros_controllers to the param server -->
  <rosparam file="$(find custom_moveit)/config/controller.yaml"/>
</launch>

The moveit_rviz.launch has to be modified according to:

In [ ]:
<launch>
  <arg name="debug" default="false" />
  <arg unless="$(arg debug)" name="launch_prefix" value="" />
  <arg     if="$(arg debug)" name="launch_prefix" value="gdb --ex run --args" />
  <arg name="rviz_config" default="$(find custom_moveit)/launch/moveit.rviz" />
  <arg     if="$(eval rviz_config=='')" name="command_args" value="" />
  <arg unless="$(eval rviz_config=='')" name="command_args" value="-d $(arg rviz_config)" />
  <node name="$(anon rviz)" launch-prefix="$(arg launch_prefix)" pkg="rviz" type="rviz" respawn="false"
        args="$(arg command_args)" output="screen">
        <rosparam command="load" file="$(find custom_moveit)/config/kinematics.yaml"/>
  </node>
</launch>

Finally, we create a new file named "final.launch". That control the robot on both simulations programs.

In [ ]:
<launch>
    <include file="$(find custom_robotic_arm)/launch/arm_gazebo.launch" />
    <include file="$(find custom_moveit)/launch/move_group.launch" />
    <include file="$(find custom_moveit)/launch/moveit_rviz.launch">
        <arg name="config" value="true"/>
    </include>
</launch>

In [ ]:
roslaunch custom_moveit final.launch

![Getting Started](./Images/1_arm_moveit_gazebo.png)

Great! congratulations!

## 1.4. Robot arm control
We create a control Package:

In [ ]:
catkin_create_pkg custom_arm_control rospy std_msgs moveit_core moveit_visual_tools moveit_ros_planning_interface tf2_geometry_msgs